本文部分内容来自西土城的搬砖日常

## 摘要
论文主要工作就是非负了LSTM的拓扑结构，由我们所熟悉的线性LSTM结构衍生出了树形LSTM结构，从而更方便地结合依存关系、短语构成等语法特性，使得语义表达更加准确。论文在预测两个句子的语义相关性（SICK）和情感分类（Stanford Sentiment treeback）两个任务上做了实验，验证了新LSTM模型的有效性。<br>
作者针对两种常见的语法树结构，Constituency tree(短语结构树)和Dependency tree(依存树)提出了两种Tree LSTM结构：Child-Sum Tree-LSTM和N-ary Tree-LSTM.<br>
![tree LSTM](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/tree-LSTM1.png)
常见的用于给句子或者短语建模的方法可以被归为三类：
- bag-of-words models(Landauer and Dumais, 1997; Folty et al. 1998): 不依赖与次序
- sequence models(Elman, 1990; Mikolov, 2012): 依赖于词序
- tree-strutured models(Coller and Kuchler, 1996; Socher et al. 2011): 依赖于给定的句子结构

## 介绍
一般的LSTM是tree-LSTM的一种特例。一般的LSTM的隐藏层输出由上一个时间点的隐藏层的输出了当前时间点的输入决定。而tree-LSTM隐藏层的输出由输入的向量和任意多的子节点的隐藏层输出决定（对于不同的句子，结构是固定的吗？？怎么确定那个时间点有哪些输入？？）。标准的LSTM可以看成是内部节点均仅有唯一一个子节点的情况。<br>
代码见于：https://github.com/stanfordnlp/treelstm
## 常见的LSTM变形
#### Bidirectional LSTM
由两个LSTM的输出concatenation而得。两个LSTM的输入分别是正常的input序列和正常input序列的倒序的序列。这使得隐藏层可以同时获得过去和将来的信息
#### Multilayer LSTM
同一时间点，第l层的隐藏层输出作为第l+1层的输入。通过这种方法使得高层的隐藏层可以获得更加久远依赖。（The idea is to let the higher layers capture longer term dependencies of the input sentence???这个点没有明白是怎么回事Graves et al., 2013; Sutskever et al., 2014; Zaremba and Sutskever, 2014）
## tree-structured LSTMs
Tree LSTM和standard LSTM的最基本的区别是，Tree LSTM的输入可以来自任意多个子节点。相应的tree LSTM中每个节点j，针对其每个Child k都含有一个forget gate：$f_{jk}$。这使得Tree LSTM隐藏层节点可以有选择的从其子节点中获取信息。<br>
和Standard LSTM一样，每个tree-LSTM单元接收一个输入向量$x_j$。其中每个节点的输入取决于树形结构的类型。比如描述依赖树的tree-LSTM，其每个节点的输入就是对应的head word。（For instance, in a Tree-LSTM over a dependency tree, each node in the tree takes the vector corresponding to the head word as input, whereas in a Tree-LSTM over a constituency tree, the leaf nodes take the corresponding word vectors as input.）这个表示不是很了？？？
#### Child-Sum Tree-LSTMs
给定一颗树，使得C(j)表示节点j的子节点集。
![Child sum tree lstms](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/child-sum_tree-LSTMs.png)
#### N-ary Tree-LSTMs
先看一下公式：
![N-ary Tree-LSTMs](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/n-ary_Tree-LSTMs.png)
其中$h_{jk},c_{jk}$便是节点j的第k个子节点的hidden state和menory cell。和前面的公式进行对比可以发现以下两点不同：
1. 子节点的数目限制为N
2. 前面的Tree LSTM中，每个forget门都由输入和对应分支的子节点决定，而其他门维持不变。然而在这里每个门，无论是forget gate(each)还是output gate ，input gata，都由该单元的输入和所有的子节点的输出决定。

论文里把这种方法夸到天上去了，在此表示不理解？？？？？？？？？<br>
The introduction o separate parameter matrices for each child k allows the N-ary Tree-LSTM model to learn more fine-grained conditioning on the states of a unit's children than the Cild sum Tree-LSTM. Consider, for example, a constituency tree application where the left child of a node corresponds to a noun phrase, and the right child to a verb phrase. Suppose that in this case it is advantageous to emphasize the verb phrase in the representation. Then the $U^{(f)}_{kl}$ parameters can be trained such that the components of $f_{j1}$are close to 0 (i.e., forget), while the components of $f_{j2}$ are close to 1.(也就是说，可以根据所有子节点的信息来判断短语的类型，并由此有决定各个子节点的影响程度。相较于前面的，其优点在于可以综合所有子节点的信息（使相互影响），而不是只能单独的查看各个子节点的信息。)<br>
另外，论文中还强调了子节点的有序性，在这里表示没有明白它体现在哪里？？？？

下面是一个具体的模型
## Semantic Relatedness of Sentence Pairs
给定两个句子，我们想要通过给出一个score来描述，这两个句子的相似性。<br>
首先，我们使用Tree-LSTM为每个句子生成对应的代表向量$h_L,h_R$。通过这两个向量，我们神经元网络计算这两个句子的相似性y。我们从距离和角度两个方面对此做出判断：(1,2,...,K对应各个评分，数值越高，表示越相似)
![semantic relatedness of Sentence Pairs](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/semantic_Relatedness_of_sentence_pairs.png)


论文剩下的部分好像也挺有趣的，有时间再看吧。